<a href="https://colab.research.google.com/github/miczkejedrzej/MNLP-project-1/blob/main/01_Data_collection_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import moduls

In [1]:
!pip install wikidata --quiet
!pip install datasets --quiet


import pandas as pd
import requests
from wikidata.client import Client
import time
from tqdm import tqdm
from google.colab import files

from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

df_train = pd.read_csv('/content/drive/MyDrive/France_Poland_multilanguage_shared_folder/[MNLP 2025 HW1] train set [PUBLIC] - train_cleaned.tsv',sep='\t')
df_dev = pd.read_csv('/content/drive/MyDrive/France_Poland_multilanguage_shared_folder/validation_df.csv', index_col=0)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-

# Function to extract the text

Functions to extract the Wikipedia text for a given item

In [2]:
def extract_entity_id(url):
    return url.strip().split("/")[-1]

def get_wikipedia_extract(wikidata_url):
    """return the Wikipedia text for a given wikidata url"""

    entity_id = extract_entity_id(wikidata_url)

    client = Client()
    try:
        item = client.get(entity_id, load=True)
        # Get english wikipedia sitelink
        sitelinks = item.data.get("sitelinks", {})
        enwiki = sitelinks.get("enwiki")

        if enwiki:
            title = enwiki["title"]

            #Wikipedia API to get the text
            api_url = "https://en.wikipedia.org/w/api.php"
            headers = {'User-Agent': 'Mozilla/5.0'}
            params = {
                "action": "query",
                "prop": "extracts",
                "explaintext": True,
                "titles": title,
                "format": "json",
                "redirects": 1
            }

            res = requests.get(api_url, params=params, headers=headers).json()
            page = next(iter(res["query"]["pages"].values()))
            return page.get("extract", "No extract found.")[:1000] #1000 caracters
        else:
            return "No English Wikipedia article found for this entity."
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 404:
            print(f"Entity not found for URL: {wikidata_url}")
            return "Entity not found in Wikidata."
        else:
            raise e  # Re-raise other HTTP errors
    except Exception as e:
        # Catch any other exception that may have occurred during Wikidata entity retrieval
        print(f"An unexpected error occurred for URL: {wikidata_url}. Error: {e}")
        return "Error retrieving Wikidata entity."

# Application to the train dataset

In [ ]:
# Application to the dataset
tqdm.pandas()
df_train['text'] = df_train['item'].progress_apply(get_wikipedia_extract)

 33%|███▎      | 2060/6251 [12:41<17:40,  3.95it/s]

An unexpected error occurred for URL: http://www.wikidata.org/entity/Q7551241. Error: HTTP Error 404: Not Found


 61%|██████    | 3812/6251 [23:41<32:53,  1.24it/s]

In [ ]:
# Checking item without text
df_train[(df_train['text'] == 'Error retrieving Wikidata entity.') |
          (df_train['text'] == "No English Wikipedia article found for this entity.")]

#Application to the dev set

In [ ]:
# Application to the dev set
df_dev['text'] = df_dev['item'].progress_apply(get_wikipedia_extract)

In [ ]:
# Checking item without text
df_dev[(df_dev['text'] == 'Error retrieving Wikidata entity.') |
          (df_dev['text'] == "No English Wikipedia article found for this entity.")]

# Exportation

In [ ]:
df_train.to_json('train_with_text.json', orient='records', lines=True)
df_dev.to_json('dev_with_text.json', orient='records', lines=True)

from google.colab import files

files.download('train_with_text.json')
files.download('dev_with_text.json')